## Building a Recommendation Engine with Collaborative Filtering

### What is Collaborative Filtering ?

Collaborative filtering is the most common technique to **give better recommendations** as more information about users is collected.Most websites like Amazon, YouTube, and Netflix use collaborative filtering as a part of their sophisticated recommendation systems.

### How does it work ?

It works by searching a large group of people and **finding a smaller set of users with tastes similar to a particular user**. It looks at the **items they like and combines them to create a ranked list of suggestions**.

### What are the different types of Collaborative Filtering ?

There are two types of Collaborative Filtering :

**`User-Based`** 

The technique where the rating matrix is used to **find similar users based on the ratings** they give, is called user-based or user-user collaborative filtering. 

**`Item-Based Collaborative Filtering`**

Using the rating matrix to **find similar items based on the ratings** given to them by users is called item-based or item-item collaborative filtering.

### What are the steps involved in Collaborative Filtering ?

To recommend items to users based on the preferences of other users , there are two steps :- 

**1. Find similar users or items to the concerned user/item.**  

**2. Predict the ratings of the items that are not yet rated by a user.**


### Key Questions 

1.How do we determine **which users or items are similar** to one another?  

2.Given that we know which users are similar, how to determine the **rating that a user would give to an item based on the ratings of similar users**?

These questions are answered in the later sections.

**Note :** The similarity is not calculated using factors like the age of users, genre of the movie, or any other data about users or items. It is calculated only on the basis of the rating (explicit or implicit) a user gives to an item.

### Importing the required libraries

In [1]:
# Basic libraries
import io
import math
import pandas as pd
import tqdm.notebook

# Surprise is a Python SciKit that comes with various recommender algorithms
# and similarity metrics to make it easy to build and analyze recommenders.
from surprise import Dataset
from surprise import Reader
from surprise import get_dataset_dir
from surprise import SVD
from surprise import KNNWithMeans
from surprise.model_selection import GridSearchCV

### Building a sample dataframe

In [2]:
# Creating a dictionary of user , item and ratings
ratings_dict = {"user": ['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D', 'E'],
                "item": [1, 2, 1, 2, 1, 2, 1, 2, 1],
                "rating": [1, 2, 2, 4, 2.5, 4, 4.5, 5, 3]}

# Building a dataframe using the dictionary ratings_dict
df = pd.DataFrame(ratings_dict)

# Inspecting the dataframe
df

,user,item,rating
0,A,1,1.0
1,A,2,2.0
2,B,1,2.0
3,B,2,4.0
4,C,1,2.5
5,C,2,4.0
6,D,1,4.5
7,D,2,5.0
8,E,1,3.0


Goal is to to to **determine the rating that user E would give to an item 2 based on the ratings of similar users**. If the rating predicted is high we can recommend that movie to E.  

### Surprise Modules and Classes

In [3]:
# Reader class is used to parse a file containing ratings.
reader = Reader()

# Loading Pandas dataframe
data = Dataset.load_from_df(df , reader)

# Building the training set
trainingSet = data.build_full_trainset()

### Collaborative filtering : Memory Based Approach

Objective : To find the **rating R that a user U would give to an item I**.

#### Answering Key Questions :

**`Q1 : Finding users similar to U who have rated the item I`**.

User    |   Ratings  
A       =   [1, 2]  
B       =   [2,4]  
C       =   [2.5, 4]  
D       =   [4.5, 5]  

a) **Euclidean Distance**

Find the distance between two points where each point represents a user and is plotted against the ratings they gave to two movies.Less the distance,more the similarity.

`Issue with Euclidean Distance` :  

**Not able to detect patterns** : After computing the distance , one would say that C is closer to D in terms of distance. But looking at the rankings, it would seem that the choices of C would align with that of A more than D because both A and C like the second movie almost twice as much as they like the first movie, but D likes both of the movies equally. 

b) **Cosine Similarity**

Consider the angle between the lines joining the origin to the respective points.

Lower Angle --> Smaller Distance --> Higher Similarity   
Higher Angle --> Higher Distance --> Lower Similarity

*`Why using cos(Angle)?`*

Because for lower angles cos gives high values (close to 1) and for higher angles (close to 0).

`Issue with Simple Cosine` :

After computing cosine values , **Users A and B are absolutely similar in the cosine similarity metric despite having different ratings** despite A being a tough rater and B being a average rater.

`Soln` : 

To factor in such individual user preferences, we will bring **all users to the same level by removing their biases** by subtracting the average rating given by that user to all items from each item rated by that user. Ex :- For user A, the rating vector [1, 2] has the average 1.5. Subtracting 1.5 from every rating would give you the vector [-0.5, 0.5].

**The cosine of the angle between the adjusted vectors is called centered cosine**. This approach is normally used when there are a lot of missing values in the vectors.

**`Q2 : Calculating the rating R based the ratings of similar users.`**

a) **Simple average approach**

We predict a user’s rating R for an item I by **taking average of the ratings given to I by the top 5 or top 10 users most similar to U.**

`Issue with simple average` : Simple average performs poorly when **there are n similar users that are not equally similar to the target user U.**   
Ex : The top 3 of them might be very similar, and the rest might not be as similar to U as the top 3.

b) **Weighted average approach**

We multiply each rating by a similarity factor(which tells how similar the users are). The similarity factor, which would act as weights, is the inverse of the distance (less distance implies higher similarity).With a weighted average, we give more consideration to the ratings of similar users in order of their similarity.

#### Implementing collaborative filtering algorithm in Surprise with KNNWithMeans.

In [4]:
# Using user-based cosine similarity
similarity_options = {"name": "cosine","user_based": True} 

# Instantiation of the KNNWithMeans collaborative filtering algorithm 
# It takes into account the mean ratings of each user.
algo = KNNWithMeans(sim_options=similarity_options)

# Fitting the training dataset
algo.fit(trainingSet)

# Predicting the rating user E would give to movie 2
prediction = algo.predict('E', 2)
prediction.est

Computing the cosine similarity matrix...
Done computing similarity matrix.


3.625

### Collaborative Filtering : Model Based Approach

This approach involves a step **to reduce or compress the large but sparse user-item matrix** as reducing dimensions can improve the performance of the algorithm in terms of both space and time.

**`Dimensionality reduction using Matrix Vectorization`** 

In the case of matrices, a matrix A with dimensions m x n can be reduced to a product of two matrices X and Y with dimensions m x p and p x n respectively.**The reduced matrices actually represent the users and items individually.** The m rows in the first matrix represent the m users, the p columns tells about the characteristics of the users and n columns represent the n items.

Consider a rating of 4 given by user C for movie 2.The rating 4 is reduced or factorized into:

**1. An user vector** (2, -1)  
**2. An item vector** (2.5, 1)

The user vector (2, -1) might represent a user C who likes horror movies and rates them positively and dislikes movies that have romance and rates them negatively.The movie 2 has a Horror rating of 2.5 and a Romance rating of 1. Multiplying it by the user vector using matrix multiplication rules gives us (2 * 2.5) + (-1 * 1) = 4.

**The columns(=p) in the user matrix and the rows (=p) in the item matrix are called latent factors** and are an indication of hidden characteristics about the users or the items.In the above example, we had two latent factors for movie genres.

One of the **popular algorithms to factorize a matrix is the singular value decomposition (SVD) algorithm.**

In [5]:
# Instantiation of Singular value decomposition algorithm 
algo = SVD(n_epochs = 10, lr_all = 0.005, reg_all = 0.4)

# Fitting the training dataset
algo.fit(trainingSet)

# Predicting the rating user E would give to movie 2
prediction = algo.predict('E', 2)
prediction.est

3.1493364783403224

# Recommendation on MovieLens Dataset

In [6]:
# Loading the builtin Movielens-100k data
movielens = Dataset.load_builtin('ml-100k')

In [7]:
# Creating a dataframe from the movielens dataset
movielens_df = pd.DataFrame(movielens.raw_ratings , columns = ['User_ID' , 'Movie_ID' , 'Ratings' , 'Timestamp'])

In [8]:
# Inspecting the movielens DataFrame
movielens_df.head()

,User_ID,Movie_ID,Ratings,Timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [9]:
# Checking the no of unique movies in the dataset
unique_movies_list = list(movielens_df['Movie_ID'].unique())

In [10]:
# Checking the no of unique users in the dataset
unique_userid_list = list(movielens_df['User_ID'].unique())

### Collaborative Filtering Using Memory Based Approach

In [11]:
# Defining the list of parameters for grid search CV
sim_options = {"name": ["msd", "cosine"],"min_support": [3, 4, 5],"user_based": [False, True]}
param_grid = {"sim_options": sim_options}

# Instantiation of Grid Search CV using the KNNWithMeans algorithm 
KNN_gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)

# Fitting the dataset to the KNN gridsearch object
KNN_gs.fit(movielens)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [12]:
# Prinitng the best RMSE score obtained
print(KNN_gs.best_score["rmse"])

# Printing the list of optimal parameters 
print(KNN_gs.best_params["rmse"])

0.9438482264734501
{'sim_options': {'name': 'msd', 'min_support': 3, 'user_based': False}}


In [13]:
# Building the training set
trainingSet = movielens.build_full_trainset()

# Instantiation of KNNWithMeans algorithm with the optimal paramters 
# obtained from Grid Search Cross Validation
KNN_algo = KNNWithMeans(name = KNN_gs.best_params["rmse"]['sim_options']['name']
                      , min_support = KNN_gs.best_params["rmse"]['sim_options']['min_support']
                      , user_based = KNN_gs.best_params["rmse"]['sim_options']['user_based'])

# Training the algorithm
KNN_algo.fit(trainingSet)

# Predicting the ratings for a particular user and movie 
KNN_algo.predict(str(196) , str(242) , r_ui=3, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
user: 196        item: 242        r_ui = 3.00   est = 3.79   {'actual_k': 40, 'was_impossible': False}


Prediction(uid='196', iid='242', r_ui=3, est=3.7940502163439223, details={'actual_k': 40, 'was_impossible': False})

In [14]:
# Checking the RMSE value on the fitted training set

error_list = []

trainset = trainingSet.build_testset()

# Iterating over all the elements of training set
for i in tqdm.notebook.tqdm(range(len(trainset)) , desc = 'outer_loop' , leave = True):
    
    # Computing the prediction for each combination of user and movie
    prediction = KNN_algo.predict(trainset[i][0] , trainset[i][1])
    
    # Calculating the error between the actual and predicted value
    error = pow(prediction.est - float(trainset[i][2]),2)
    
    # Storing each of the squared error value in a list
    error_list.append(error)
    
# Printing the RMSE value after computing mean and taking the root of MSE     
print("RMSE : {}" .format(pow(sum(error_list)/len(error_list) , 1/2)))


RMSE : 0.7727877044627107


In [15]:
movies_user_might_like = []

# Iterating over the no of unique users for personalized recommendation
for i in tqdm.notebook.tqdm(range(len(unique_userid_list)) , desc = 'outer_loop' , leave = True):
    
    # Determining for which movies the user has already provided the ratings
    # Effectively determining which movies user has watched
    movie_ratings_to_be_predicted = None
    movies_watched_by_user = list(movielens_df[movielens_df['User_ID'] == unique_userid_list[i]]['Movie_ID'].unique())
    
    # Generating list of the movies that can be provided as a possible recommendation
    movie_ratings_to_be_predicted = list(set(unique_movies_list) - set(movies_watched_by_user))
    
    # Looping through each of the movies that can be recommended
    for j in range(len(movie_ratings_to_be_predicted)):
        
        # Getting the movie_id stored
        movie_id , prediction = None , None
        movie_id = movie_ratings_to_be_predicted[j]
        
        # Generation of the prediction for each combination of the user and movie
        prediction = KNN_algo.predict(unique_userid_list[i] , movie_id).est
        
        # Storing the predicted rating for each combination of the user and movie in a list
        movies_user_might_like.append([unique_userid_list[i] , movie_id , prediction])

In [16]:
# Creating a dataframe sorted by Ratings and grouped by User ID to fetch the top 10 recommendation for an user

columns = ['User_ID' , 'Reco_Movie_ID' , 'Predicted_Rating']
KNN_recommended_movies_df = pd.DataFrame(movies_user_might_like , 
                                     columns = columns).sort_values(['User_ID' ,'Predicted_Rating'],
                                     ascending=False).groupby('User_ID').head(10)

In [17]:
def read_item_names():
    """Reading the u.item file from MovieLens 100-k dataset 
    and returning a mapping to convert raw ids into movie names.
    """

    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'     # Reading the u.item file from MovieLens 100-k dataset
    rid_to_name = {}                                              # Creating a dictionary for mapping row id to movie name  
    
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:     # Opening the file in read mode
        
        for line in f:                                            # Iterating through each line
            line = line.split('|')                                # Using pipe as delimiter
            rid_to_name[line[0]] = line[1]                        # Mapping of row id and movie name

    return rid_to_name

# Read the mappings raw id <-> movie name
rid_to_name = read_item_names()

# Inspecting the created dictionary
rid_to_name

{'1': 'Toy Story (1995)',
 '2': 'GoldenEye (1995)',
 '3': 'Four Rooms (1995)',
 '4': 'Get Shorty (1995)',
 '5': 'Copycat (1995)',
 '6': 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 '7': 'Twelve Monkeys (1995)',
 '8': 'Babe (1995)',
 '9': 'Dead Man Walking (1995)',
 '10': 'Richard III (1995)',
 '11': 'Seven (Se7en) (1995)',
 '12': 'Usual Suspects, The (1995)',
 '13': 'Mighty Aphrodite (1995)',
 '14': 'Postino, Il (1994)',
 '15': "Mr. Holland's Opus (1995)",
 '16': 'French Twist (Gazon maudit) (1995)',
 '17': 'From Dusk Till Dawn (1996)',
 '18': 'White Balloon, The (1995)',
 '19': "Antonia's Line (1995)",
 '20': 'Angels and Insects (1995)',
 '21': 'Muppet Treasure Island (1996)',
 '22': 'Braveheart (1995)',
 '23': 'Taxi Driver (1976)',
 '24': 'Rumble in the Bronx (1995)',
 '25': 'Birdcage, The (1996)',
 '26': 'Brothers McMullen, The (1995)',
 '27': 'Bad Boys (1995)',
 '28': 'Apollo 13 (1995)',
 '29': 'Batman Forever (1995)',
 '30': 'Belle de jour (1967)',
 '31': 'Crimson Tide

In [18]:
# Adding a column containing the movie names
KNN_recommended_movies_df['Movie_Name'] = KNN_recommended_movies_df['Reco_Movie_ID'].apply(lambda x : rid_to_name[x])

# Inspecting the DataFrame
KNN_recommended_movies_df

,User_ID,Reco_Movie_ID,Predicted_Rating,Movie_Name
69236,99,1536,5.000000,Aiqing wansui (1994)
69620,99,1467,5.000000,"Saint of Fort Washington, The (1993)"
69868,99,814,5.000000,"Great Day in Harlem, A (1994)"
69514,99,1500,4.981619,Santa with Muscles (1996)
69263,99,1599,4.969722,Someone Else's America (1995)
...,...,...,...,...
178835,1,1642,4.918548,Some Mother's Son (1996)
178890,1,1653,4.904412,Entertaining Angels: The Dorothy Day Story (1996)
178421,1,1293,4.827150,Star Kid (1997)
178653,1,1449,4.803520,Pather Panchali (1955)


In [19]:
# Displaying what movies need to be recommended to the user with user ID 99
KNN_recommended_movies_df[KNN_recommended_movies_df['User_ID'] == '99']['Movie_Name']

69236                                 Aiqing wansui (1994)
69620                 Saint of Fort Washington, The (1993)
69868                        Great Day in Harlem, A (1994)
69514                            Santa with Muscles (1996)
69263                        Someone Else's America (1995)
69592    Entertaining Angels: The Dorothy Day Story (1996)
69534                             Some Mother's Son (1996)
69333                               Pather Panchali (1955)
69054                                      Star Kid (1997)
68802                                          Anna (1996)
Name: Movie_Name, dtype: object

In [20]:
# Displaying the highest rated 10 movies by user with user ID 99 
movielens_df[movielens_df['User_ID'] == '99'].sort_values('Ratings').tail(10)['Movie_ID'].apply(lambda x : rid_to_name[x])

44374                               Titanic (1997)
6647                                Contact (1997)
41742    Indiana Jones and the Last Crusade (1989)
12800                   Usual Suspects, The (1995)
7502                          Seven (Se7en) (1995)
8781                           Pulp Fiction (1994)
9910                                  Fargo (1996)
9294                          Jerry Maguire (1996)
56992                         Dumb & Dumber (1994)
48                               Get Shorty (1995)
Name: Movie_ID, dtype: object

### Collaborative Filtering Using Model Based Approach

In [21]:
# Defining the list of parameters for grid search CV
param_grid = {"n_epochs": [5, 10],"lr_all": [0.002, 0.005],"reg_all": [0.4, 0.6]}

# Instantiation of Grid Search CV using the SVD algorithm 
svd_gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

# Fitting the dataset to the SVD gridsearch object
svd_gs.fit(movielens)

In [22]:
# Prinitng the best RMSE score obtained
print(svd_gs.best_score["rmse"])

# Printing the list of optimal parameters 
print(svd_gs.best_params["rmse"])

0.9643565653533376
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [23]:
# Instantiation of SVD algorithm with the optimal paramters 
# obtained from Grid Search Cross Validation
svd_algo = SVD(n_epochs = svd_gs.best_params["rmse"]['n_epochs']
               , lr_all = svd_gs.best_params["rmse"]['lr_all'], 
               reg_all = svd_gs.best_params["rmse"]['reg_all'])

# Training the algorithm
svd_algo.fit(trainingSet)

# Predicting the ratings for a particular user and movie
svd_algo.predict(str(196) , str(242) , r_ui=3, verbose=True)

user: 196        item: 242        r_ui = 3.00   est = 3.88   {'was_impossible': False}


Prediction(uid='196', iid='242', r_ui=3, est=3.881516705938398, details={'was_impossible': False})

In [24]:
# Checking the RMSE value on the fitted training set
trainset = trainingSet.build_testset()

error_list = []

# Iterating over all the elements of training set
for i in tqdm.notebook.tqdm(range(len(trainset)) , desc = 'outer_loop' , leave = True):
    
    # Computing the prediction for each combination of user and movie
    prediction = svd_algo.predict(trainset[i][0] , trainset[i][1])
    
    # Calculating the error between the actual and predicted value
    error = pow(prediction.est - float(trainset[i][2]),2)
    
    # Storing each of the squared error value in a list
    error_list.append(error)
    
# Printing the RMSE value after computing mean and taking the root of MSE
print("RMSE : {}" .format(pow(sum(error_list)/len(error_list) , 1/2)))


RMSE : 0.9401903111900936


In [25]:
movies_user_might_like = []

# Iterating over the no of unique users for personalized recommendation
for i in tqdm.notebook.tqdm(range(len(unique_userid_list)) , desc = 'outer_loop' , leave = True):
    
    # Determining for which movies the user has already provided the ratings
    # Effectively , determining which movies user has watched
    movie_ratings_to_be_predicted = None
    movies_watched_by_user = list(movielens_df[movielens_df['User_ID'] == unique_userid_list[i]]['Movie_ID'].unique())
    
    # Generating list of the movies that can be provided as a possible recommendation
    movie_ratings_to_be_predicted = list(set(unique_movies_list) - set(movies_watched_by_user))
    
    # Looping through each of the movies that can be recommended
    for j in range(len(movie_ratings_to_be_predicted)):
        
        # Getting the movie_id stored
        movie_id , prediction = None , None
        movie_id = movie_ratings_to_be_predicted[j]
        
        # Generation of the prediction for each combination of the user and movie
        prediction = svd_algo.predict(unique_userid_list[i] , movie_id).est
        
        # Storing the predicted rating for each combination of the user and movie in a list
        movies_user_might_like.append([unique_userid_list[i] , movie_id , prediction])

In [26]:
# Creating a dataframe sorted by Ratings and grouped by User ID to fetch the top 10 recommendation for an user
columns = ['User_ID' , 'Reco_Movie_ID' , 'Ratings']

SVD_recommended_movies_df = pd.DataFrame(movies_user_might_like , 
                                     columns = columns).sort_values(['User_ID' , 'Ratings'],
                                     ascending=False).groupby('User_ID').head(10)

In [27]:
# Adding a column containing the movie names
SVD_recommended_movies_df['Movie_Name'] = SVD_recommended_movies_df['Reco_Movie_ID'].apply(lambda x : rid_to_name[x])

# Inspecting the DataFrame
SVD_recommended_movies_df

,User_ID,Reco_Movie_ID,Ratings,Movie_Name
68799,99,408,4.341233,"Close Shave, A (1995)"
68792,99,169,4.311158,"Wrong Trousers, The (1993)"
69607,99,318,4.301746,Schindler's List (1993)
70116,99,114,4.247337,Wallace & Gromit: The Best of Aardman Animatio...
69523,99,483,4.244852,Casablanca (1942)
...,...,...,...,...
179071,1,480,4.143326,North by Northwest (1959)
179196,1,357,4.138032,One Flew Over the Cuckoo's Nest (1975)
179101,1,657,4.121034,"Manchurian Candidate, The (1962)"
178265,1,515,4.108803,"Boot, Das (1981)"


In [28]:
# Displaying what movies need to be recommended to the user with user ID 405
SVD_recommended_movies_df[SVD_recommended_movies_df['User_ID'] == '405']['Movie_Name']

631732                                Close Shave, A (1995)
632399    Wallace & Gromit: The Best of Aardman Animatio...
631639                                    Casablanca (1942)
632257                             Good Will Hunting (1997)
631798                                  Citizen Kane (1941)
632417                               Shall We Dance? (1996)
632255                                Secrets & Lies (1996)
632179    Dr. Strangelove or: How I Learned to Stop Worr...
631994              Some Folks Call It a Sling Blade (1993)
632540                                       Vertigo (1958)
Name: Movie_Name, dtype: object

In [29]:
# Displaying the highest rated 10 movies by user with user ID 405
movielens_df[movielens_df['User_ID'] == '405'].sort_values('Ratings').tail(10)['Movie_ID'].apply(lambda x : rid_to_name[x])

71258            Man Without a Face, The (1993)
71023                        French Kiss (1995)
70739                       Philadelphia (1993)
70648                         Cinderella (1950)
69953            Godfather: Part II, The (1974)
31627              To Kill a Mockingbird (1962)
32238                              Glory (1989)
65791                   Another Stakeout (1993)
78214    One Flew Over the Cuckoo's Nest (1975)
74480    Snow White and the Seven Dwarfs (1937)
Name: Movie_ID, dtype: object